In [1]:
from classiq import *

size = 4

@qfunc
def prep_minus(x: QBit):  # preparing the |-> state
  X(x)
  H(x)

@qfunc
def diffuser(x: QNum, aux: Output[QNum]):  # the condition of the diffuser
  aux ^= (x != 0)

@qfunc
def zero_diffuser(x: QNum):  # applying the zero diffuser and reversing it (like Grover algo) with within_apply
  aux = QNum('aux')
  allocate(1, aux)
  within_apply(compute=lambda: prep_minus(aux), action=lambda: diffuser)

def C_iter(i: int, vertices: QNum, adjacents: QNum):  # the Coin operator's iteration cycle
  probs = [0] * (2 ** size)  # creating a list of probabilities for each node

  if i == 0:
    probs[i+1] = 1.0  # if the walker is at node 0 (first node), it can only go right, therefore setting the probability of the right node (1) to 100%

  elif i == 15:
    probs[i-1] = 1.0  # if the walker is at node 15 (last node), it can only go left, therefore setting the probability of the left node (14) to 100%

  else:               # the other nodes in the middle have equal probabilities to go either right or left (50%)
    probs[i+1] = 0.5
    probs[i-1] = 0.5

  print(f'State = {i}, prob vec = {probs}')  # checking out the probs list for each state

  # we now apply a controlled transform (controlled zero diffuser) onto the walker's state based on the probabilities
  control(ctrl= vertices==i,
          operand= lambda: within_apply(compute= lambda: inplace_prepare_state(probabilities=probs, bound=0.01, target=adjacents),
                                        action= lambda: zero_diffuser(adjacents))
          )

@qfunc
def C_operator(vertices: QNum, adjacents: QNum):  # the Coin operator which will repeat the Coin iteration as many times as the number of nodes
  for i in range(2**size):
    C_iter(i, vertices, adjacents)
    
@qfunc
def edge(res: Output[QBit], vertices: QNum, adjacents: QNum):  # an edge-checking function to see which nodes are adjacent
  res |= (((vertices + adjacents) % 2) == 1)  # this condition is general to most quantum walks, including this problem

@qfunc
def bit_swap(x: QArray[QBit], y: QArray[QBit]):  # the bitwise swap operation applies repeatedly to swap the nodes, based on the probabilities from the Coin operation
  repeat(count= x.len,
         iteration= lambda i: SWAP(x[i], y[i]))

@qfunc
def S_operator(vertices: QNum, adjacents: QNum):  # the Shift operator which first does the edge-checking, then applies the bit-swap if the nodes are adjacent
  res = QNum('res')
  edge(res, vertices, adjacents)
  control(ctrl= res==1,
          operand= lambda: bit_swap(vertices, adjacents))

@qfunc
def main(vertices: Output[QNum], adjacents: Output[QNum]):
  allocate(size, vertices)
  hadamard_transform(vertices)  # equal superposition of vertices
  allocate(size, adjacents)

  C_operator(vertices, adjacents)  # applying the Coin operation
  S_operator(vertices, adjacents)  # applying the Shift operation

qmodel = create_model(main)
qprogram = synthesize(qmodel)
show(qprogram)    

State = 0, prob vec = [0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State = 1, prob vec = [0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State = 2, prob vec = [0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State = 3, prob vec = [0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State = 4, prob vec = [0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State = 5, prob vec = [0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State = 6, prob vec = [0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State = 7, prob vec = [0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State = 8, prob vec = [0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State = 9, prob vec = [0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State = 10, prob vec = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State = 11, prob vec = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State = 12, prob vec = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State = 13, prob vec = [

ValidationError: 8 validation errors for GlobalVersions
deprecated -> 0.43.0 -> deprecation_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.0 -> removal_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.1 -> deprecation_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.1 -> removal_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.2 -> deprecation_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.2 -> removal_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.3 -> deprecation_date
  invalid datetime format (type=value_error.datetime)
deprecated -> 0.43.3 -> removal_date
  invalid datetime format (type=value_error.datetime)